# Voting, Stacking and Blending

In [1]:
%pylab inline
import numpy as np
import pandas as pd
import pylab as plt

import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Задача регрессии

In [2]:
from sklearn.datasets import load_boston

data = load_boston()

X_full = data.data
y_full = data.target

In [3]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size=100, 
                                        random_state=241)
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=0.35, 
                                        random_state=241)

In [4]:
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(Xtrain.shape[0], shuffle=True, random_state=241)

In [5]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
print(cross_val_score(rf, Xtrain, ytrain, cv=cv, scoring ='neg_mean_squared_error').mean())

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


-15.156913688212926


In [6]:
from sklearn.metrics import mean_squared_error

rf.fit(Xtrain, ytrain)
pred_rf = rf.predict(Xval)
mean_squared_error(pred_rf, yval)

10.955697902097905

In [7]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor()
print(cross_val_score(ada, Xtrain, ytrain, cv=cv, scoring ='neg_mean_squared_error').mean())

-17.380011927514122


In [8]:
ada.fit(Xtrain, ytrain)
pred_ada = ada.predict(Xval)
mean_squared_error(pred_ada, yval)

13.011518542332666

# Blending

1) смешаем алгоритмы с весами 1:1, то есть в качестве ответа предскажем среднее значение смеси

In [9]:
pred_mix_1 = 0.5 *(pred_rf + pred_ada)
print('Validation prediction:',mean_squared_error(pred_mix_1, yval))

rf_test = rf.predict(X_test)
ada_test = ada.predict(X_test)
pred_test_1 = 0.5 *(rf_test + ada_test)
print('Test prediction:',mean_squared_error(pred_test_1, y_test))

Validation prediction: 10.241624814300708
Test prediction: 9.670834022421047


2) Смешаем алгоритмы с весами $\alpha$ : $1-\alpha$. Подберём $\alpha$ в цикле

In [10]:
max_score = -100
opt_alpha = 0

for alpha in np.arange(0.01,1.01,0.01):
    pred_mix_2 = alpha * pred_rf + (1-alpha) * pred_ada
    curr_score = -mean_squared_error(pred_mix_2, yval)
    
    if curr_score > max_score:
        max_score = curr_score
        opt_alpha = alpha
        
print(opt_alpha, -max_score)

0.65 10.090030224977806


In [11]:
rf_test = rf.predict(X_test)
ada_test = ada.predict(X_test)
pred_test_2 = opt_alpha * rf_test + (1-opt_alpha) * ada_test
print('Test prediction:',mean_squared_error(pred_test_2, y_test))

Test prediction: 10.060504460467717


# Stacking

In [12]:
SecondTrain = pd.DataFrame(np.nan, index=np.arange(len(Xval)), columns=['RF','ADA'])

SecondTrain['RF'] = rf.predict(Xval)
SecondTrain['ADA'] = ada.predict(Xval)

In [13]:
meta = RandomForestRegressor()
meta.fit(SecondTrain,yval)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [14]:
rf_test = rf.predict(X_test)
ada_test = ada.predict(X_test)

FirstLevelPred = pd.DataFrame(np.nan, index=np.arange(len(X_test)), columns=['RF','ADA'])

FirstLevelPred['RF'] = rf.predict(X_test)
FirstLevelPred['ADA'] = ada.predict(X_test)

In [15]:
final_pred = meta.predict(FirstLevelPred)

print('Final prediction:',mean_squared_error(final_pred,y_test))

Final prediction: 11.358251999999998


# Задача классификации

In [16]:
from sklearn.datasets import load_iris

data = load_iris()

X_full = data.data
y_full = data.target

In [17]:
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size=100, 
                                        random_state=135)

In [18]:
from sklearn.model_selection import KFold, cross_val_score
cv = KFold(X.shape[0], shuffle=True, random_state=123)

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
print(cross_val_score(rf, X, y, cv=cv, scoring ='accuracy').mean())

0.96


In [20]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
print(cross_val_score(lr, X, y, cv=cv, scoring ='accuracy').mean())

0.88


# Voting

In [21]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

voting = VotingClassifier(estimators=[('RF',rf),('LR',lr)], voting='soft')
voting.fit(X, y)
pred_voting = voting.predict(X_test)

print('Accuracy on test:', accuracy_score(pred_voting,y_test))

Accuracy on test: 0.96


/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Задание

1) Разбейте X, y на тренировочную и валидационную части

2) Сделайте предсказание с помощью блендинга (предсказывайте вероятности классов, а не сами классы - методом predict_proba)

3) Сделайте предсказание с помощью стекинга (предсказывайте вероятности классов)

**Blending**

In [ ]:
#Your code here
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=135)
pred_proba_rf = rf.predict_proba(X_test)
pred_proba_lr = lr.predict_proba(X_test)

In [ ]:
mix_pred = 0.5 * (pred_proba_rf + pred_proba_lr)

pred_final = [np.argmax(elem) for elem in mix_pred]

print('Accuracy blending:', accuracy_score(pred_final, y_test))

**Stacking**

In [ ]:
#Your code here

# Готовое решение для stacking

Вернемся к задаче регрессии с датасетом Boston.

In [22]:
from vecstack import stacking

In [23]:
models = [RandomForestRegressor(),AdaBoostRegressor()]

S_train, S_test = stacking(models, X, y, X_test, regression=True, verbose=1)

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestRegressor]
    ----
    MEAN:     [0.07628205] + [0.02572902]
    FULL:     [0.07600000]

model  1:     [AdaBoostRegressor]
    ----
    MEAN:     [0.06089744] + [0.03525641]
    FULL:     [0.06000000]



In [24]:
meta = RandomForestRegressor()
meta.fit(S_train, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [25]:
final_pred = meta.predict(S_test)

print('Final prediction:',mean_squared_error(final_pred,y_test))

Final prediction: 0.0639387661566127


# Задание

Используйте библиотеку vecstack для решения задачи классификации Iris с помощью stacking

In [26]:
#your code here
from sklearn.datasets import load_iris
data = load_iris()
X_full = data.data
y_full = data.target
X, X_test, y, y_test = train_test_split(X_full, y_full, test_size=100, random_state=135)
models = [LogisticRegression(), RandomForestClassifier()]
S_train, S_test = stacking(models, X, y, X_test, regression=True, verbose=1)

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [LogisticRegression]
    ----
    MEAN:     [0.14583333] + [0.17052818]
    FULL:     [0.14000000]

model  1:     [RandomForestClassifier]
    ----
    MEAN:     [0.06089744] + [0.03525641]
    FULL:     [0.06000000]



In [27]:
meta = LogisticRegression()
meta.fit(S_train, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
final_pred = meta.predict(S_test)

print('Final prediction:',mean_squared_error(final_pred,y_test))

Final prediction: 0.19


# Задание

Решите задачу с данными train_medium.csv, test_medium.csv, используя stacking и/или blending нескольких алгоритмов. Сравните полученное качество (на кросс-валидации) с качеством, полученным с помощью xgboost, catboost, lightgbm в отдельности.

In [ ]:
#your code here